In [1]:
import os
import random
import sys
if os.path.dirname(os.getcwd()) not in sys.path:
    sys.path.append(os.path.dirname(os.getcwd()))
import torch
import torch.nn as nn
import torchvision

import time
import copy
import numpy as np
from torchvision import transforms
from tqdm import tqdm
from torchsummary import summary

print(f"torch = {torch.__version__}")
print(f"torchvision = {torchvision.__version__}")


torch = 1.12.1
torchvision = 0.13.1


In [2]:
from models import quat_mobilenet_v2
tiny_model = quat_mobilenet_v2(cifar10=True)
tiny_model.load_state_dict(torch.load("../models/weights/tiny_mobilenetv2_cifar.pt"))
summary(tiny_model,(3,32,32),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantStub-1            [-1, 3, 32, 32]               0
            Conv2d-2           [-1, 32, 16, 16]             864
       BatchNorm2d-3           [-1, 32, 16, 16]              64
             ReLU6-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 32, 16, 16]             288
       BatchNorm2d-6           [-1, 32, 16, 16]              64
             ReLU6-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 16, 16, 16]             512
       BatchNorm2d-9           [-1, 16, 16, 16]              32
QuantizableInvertedResidual-10           [-1, 16, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]           1,024
      BatchNorm2d-12           [-1, 64, 16, 16]             128
            ReLU6-13           [-1, 64, 16, 16]               0
           Conv2d-14         

In [3]:
min_val_cut = torch.tensor(-0.127)
max_val_cut = torch.tensor(0.127)

for name, param in tiny_model.named_parameters():
    print(param.dim())
    if param.dim()==4:
        y = param.clone()

        Min,Max = torch.aminmax(y, dim=1)
        Min = torch.min(Min,min_val_cut)
        Max = torch.max(Max,max_val_cut)

        s = (Max-Min)/(max_val_cut-min_val_cut)
        z = min_val_cut -torch.round(Min/s,decimals=3)
        
        for i in range(len(y)):
            y[i] = torch.round(y[i]/s[i]+z[i],decimals=3)
            y[i] = (y[i]-z[i])*s[i]
        param=y
    else:
        print(name,param)
        

4
1
features.0.1.weight Parameter containing:
tensor([0.5196, 0.5371, 0.2668, 0.4708, 0.2503, 0.2903, 0.4309, 0.2674, 0.5359,
        0.5124, 0.5269, 0.5179, 0.2491, 0.4390, 0.5380, 0.5384, 0.5445, 0.4856,
        0.3875, 0.3633, 0.5395, 0.5229, 0.5223, 0.4575, 0.5456, 0.5198, 0.4046,
        0.5033, 0.5404, 0.4906, 0.4406, 0.4894], requires_grad=True)
1
features.0.1.bias Parameter containing:
tensor([ 0.1546, -0.0500,  0.4752,  0.2713,  0.4899,  0.4603,  0.3403,  0.4740,
         0.0763,  0.1721,  0.1227,  0.1648,  0.4932,  0.3296,  0.0673,  0.0275,
        -0.0367,  0.2736,  0.3786,  0.4028, -0.0476,  0.1536,  0.1372,  0.3042,
         0.0187,  0.1481,  0.3765,  0.2018,  0.0308,  0.2339,  0.3379,  0.2315],
       requires_grad=True)
4
1
features.1.conv.0.1.weight Parameter containing:
tensor([0.5167, 0.5097, 0.5157, 0.4273, 0.6762, 0.6409, 0.4990, 0.5801, 0.4369,
        0.5996, 0.4290, 0.3597, 0.7089, 0.6622, 0.6199, 0.4657, 0.3474, 0.5844,
        0.4283, 0.5805, 0.3848, 0.4647, 0.

In [4]:
from utils import Data
train_loader, test_loader = Data.Cifar10_Dataloader(filepath="../data/")
for inputs, label in iter(train_loader):
    print(inputs.shape)
    for i in range(len(inputs)):
        temp = inputs[i].detach()
        print(temp)
        M = torch.max(temp)
        m = torch.min(temp)
        inputs[i] = torch.round(254*(inputs[i]-m)/(M-m)-127)/1000
        print(inputs[i])
        inputs[i] = (1000*inputs[i]+127)*(M-m)/254+m
    break

Files already downloaded and verified
Files already downloaded and verified
Train data set = 50000, Test = 10000
torch.Size([128, 3, 32, 32])
tensor([[[-0.1804,  0.1297,  0.6919,  ..., -2.4291, -2.4291, -2.4291],
         [-1.0527, -0.5100,  0.4205,  ..., -2.4291, -2.4291, -2.4291],
         [-1.5567, -0.8007, -0.5487,  ..., -2.4291, -2.4291, -2.4291],
         ...,
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291]],

        [[ 0.2761,  0.5908,  1.1218,  ..., -2.4183, -2.4183, -2.4183],
         [-0.6089, -0.0386,  0.9251,  ..., -2.4183, -2.4183, -2.4183],
         [-1.2186, -0.4122, -0.1762,  ..., -2.4183, -2.4183, -2.4183],
         ...,
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
         [-2.4183, -2.4183, -2.4183,  ..., -2.4

In [5]:
for inputs, label in iter(train_loader):
        temp = inputs.detach()
        # inputs = inputs.flatten(start_dim=1)
        print(inputs.shape, torch.min(inputs.flatten(start_dim=1)))
        M = torch.max(torch.max(inputs),torch.full_like(inputs,torch.min(inputs)))
        m = torch.min(torch.min(inputs),torch.full_like(inputs,torch.max(inputs)))
        # M = torch.max(torch.max(inputs),torch.full_like(inputs,0.127))
        # m = torch.min(torch.min(inputs),torch.full_like(inputs,-0.127)) 
        print(M.size(),m.size())
        inputs = torch.round(254*(inputs-m)/(M-m)-127) /1000 
        for i in range(len(inputs)):
                print(torch.max(inputs[i]),torch.min(inputs[i]))
        # print(inputs)
        inputs = (1000*inputs+127)*(M-m)/254+m
        print(inputs.shape)

        break

torch.Size([128, 3, 32, 32]) tensor(-2.4291)
torch.Size([128, 3, 32, 32]) torch.Size([128, 3, 32, 32])
tensor(0.1270) tensor(-0.1270)
tensor(0.1040) tensor(-0.1270)
tensor(0.0580) tensor(-0.1270)
tensor(0.1180) tensor(-0.1270)
tensor(0.1270) tensor(-0.1270)
tensor(0.1220) tensor(-0.1270)
tensor(0.1270) tensor(-0.1270)
tensor(0.1270) tensor(-0.1270)
tensor(0.0500) tensor(-0.1270)
tensor(0.1270) tensor(-0.1270)
tensor(0.1120) tensor(-0.1270)
tensor(0.1070) tensor(-0.1270)
tensor(0.1070) tensor(-0.1270)
tensor(0.0980) tensor(-0.1270)
tensor(0.1270) tensor(-0.1270)
tensor(0.1270) tensor(-0.1270)
tensor(0.1270) tensor(-0.1270)
tensor(0.0920) tensor(-0.1270)
tensor(0.1220) tensor(-0.1270)
tensor(0.1190) tensor(-0.1270)
tensor(0.1040) tensor(-0.1270)
tensor(0.1240) tensor(-0.1270)
tensor(0.1040) tensor(-0.1270)
tensor(0.0800) tensor(-0.1270)
tensor(0.1060) tensor(-0.1270)
tensor(0.0920) tensor(-0.1270)
tensor(0.1180) tensor(-0.1270)
tensor(0.1270) tensor(-0.1270)
tensor(0.0900) tensor(-0.1270

In [9]:
min_val_cut = torch.tensor(-0.127)
max_val_cut = torch.tensor(0.127)

for name, param in tiny_model.named_parameters():
    if param.dim()==4:
        y = param.detach()
        y = torch.ones_like(y)
        print(y)
        Min,Max = torch.aminmax(y, dim=1)
        Min = torch.min(Min,min_val_cut)
        Max = torch.max(Max,max_val_cut)

        s = (Max-Min)/(max_val_cut-min_val_cut)
        z = min_val_cut -torch.round(Min/s,decimals=3)

        for i in range(len(y)):
            y[i] = torch.round(y[i]/s[i]+z[i],decimals=3)
            y[i] = (y[i]-z[i])*s[i]
        print(y)
        break

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
      